In [1]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import numpy as np
import datetime
import calendar
#import datetime from datetime
import math
import re
import pint
from sklearn import linear_model
data = pd.read_csv('fifa21_train.csv')
data.shape

(11701, 101)

In [3]:
new_col = []
for col in data.columns:
    new_col.append(col.lower())
    #print(col)
data.columns = new_col
data.columns
    

Index(['id', 'name', 'age', 'nationality', 'club', 'bp', 'position',
       'team & contract', 'height', 'weight',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'ova'],
      dtype='object', length=101)

In [4]:
data.isna().sum()

id              0
name            0
age             0
nationality     0
club           18
               ..
cb              0
rcb             0
rb              0
gk              0
ova             0
Length: 101, dtype: int64

In [5]:
numeric = data.select_dtypes(include = np.number)
numeric.head()

,id,age,growth,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,...,gk reflexes,total stats,base stats,pac,sho,pas,dri,def,phy,ova
0,184383,26,1,258,54,47,43,70,44.0,286,...,6,1682,357,69,51,63,63,51,60,64
1,188044,30,0,365,66,79,76,68,76.0,375,...,16,1961,412,83,75,68,82,33,71,77
2,184431,33,0,336,73,76,34,78,75.0,424,...,3,1925,404,80,77,78,86,27,56,80
3,233796,22,13,242,44,42,58,62,36.0,259,...,12,1527,329,57,44,54,57,57,60,59
4,234799,23,8,249,49,37,61,68,34.0,280,...,15,1664,360,66,44,60,64,60,66,65


In [6]:
categorical = data.select_dtypes(include = object)
categorical.head()

,name,nationality,club,bp,position,team & contract,height,weight,foot,joined,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,A. Pasche,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,"Jul 1, 2015",...,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1
1,Alan Carvalho,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,"Jan 16, 2015",...,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2
2,S. Giovinco,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,"Jan 31, 2019",...,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2
3,J. Evans,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,"Jul 1, 2016",...,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2
4,Y. Demoncy,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,"Jul 1, 2018",...,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2


In [7]:
categorical.isna().sum()

name                   0
nationality            0
club                  18
bp                     0
position             288
team & contract        0
height                 0
weight                 0
foot                   0
joined                41
loan date end      11072
value                  0
wage                   0
release clause         0
contract               0
w/f                    0
sm                     0
a/w                   59
d/w                   59
ir                     0
hits                   0
ls                     0
st                     0
rs                     0
lw                     0
lf                     0
cf                     0
rf                     0
rw                     0
lam                    0
cam                    0
ram                    0
lm                     0
lcm                    0
cm                     0
rcm                    0
rm                     0
lwb                    0
ldm                    0
cdm                    0


In [8]:
categorical['club'].value_counts()

OGC Nice          35
Crystal Palace    35
Everton           35
Chelsea           33
Sporting CP       33
                  ..
GIF Sundsvall      1
FC Orenburg        1
Germany            1
LKS Lodz           1
Santo André        1
Name: club, Length: 885, dtype: int64

In [9]:
#FOR PLAYERS WHERE CLUBS IS NA, WE ARE CLASSIFYING THEM AS FREE AGENTS
categorical['club'] = categorical['club'].fillna('free agent')

C:\Users\KIMERA\AppData\Local\Temp/ipykernel_20324/1969520170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical['club'] = categorical['club'].fillna('free agent')


In [10]:
categorical['club'].isna().sum()

0

In [11]:
#FILL THE POSITION NaN WITH MODE
#WE SHALL CONSTRUCT A MODEL TO PREDICT THE PLAYER POSITION LATER
mode = categorical['position'].mode()[0]
mode
#LINK:https://stackoverflow.com/questions/21082671/find-and-select-the-most-frequent-data-of-column-in-pandas-dataframe

'CB'

In [12]:
#REPLACE NaN IN POSITION COLUMN WITH MODE
categorical['position'] = categorical['position'].fillna(mode)

C:\Users\KIMERA\AppData\Local\Temp/ipykernel_20324/2505189410.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical['position'] = categorical['position'].fillna(mode)


In [13]:
#CHECKING FOR NaN IN POSITION COLUMN
categorical['position'].isna().sum()

0

In [14]:
#SET NAN IN JOINED DATE TO 1ST SEPTEMBER 2020
#REASON: FiFA 21 is released usually in the second week of September or October. They usually stop collecting data on 1st September.
#So players who change clubs after that may not be updated in the game data
#SET DATE TO THE 1st DAY OF SEPTEMBER
first_day = datetime.date(2020, 9, 1)
display(first_day)
categorical['joined'] = categorical['joined'].fillna(first_day.strftime('%b %d, %Y'))

datetime.date(2020, 9, 1)

C:\Users\KIMERA\AppData\Local\Temp/ipykernel_20324/2129599596.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical['joined'] = categorical['joined'].fillna(first_day.strftime('%b %d, %Y'))


In [15]:
categorical['joined'].isna().sum()

0

In [16]:
#GETTING RATIO OF NaN TO ACTUAL VALUES IN THE 'Loan Date End' COLUMN TO LENGTH OF DF
display(categorical['loan date end'].isna().sum())
perc = (categorical['loan date end'].isna().sum()/data.shape[0]) * 100
perc

11072

94.62439107768567

In [17]:
#OVER 94% OF THE VALUES IN THE 'Loan Date End' ARE NaN. SO, WE ARE DROPPING IT
categorical = categorical.drop(['loan date end'], axis = 1)
categorical.columns

Index(['name', 'nationality', 'club', 'bp', 'position', 'team & contract',
       'height', 'weight', 'foot', 'joined', 'value', 'wage', 'release clause',
       'contract', 'w/f', 'sm', 'a/w', 'd/w', 'ir', 'hits', 'ls', 'st', 'rs',
       'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm',
       'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb',
       'rcb', 'rb', 'gk'],
      dtype='object')

In [18]:
# d = pd.Period('Dec 01, 2021',freq='M').end_time.date() 
# print(d)   

In [19]:
x = '80 + 2'
y = int(x.split('+')[0]) + int(x.split('+')[1])
y

82

In [20]:
numeric.isna().sum()

id                    0
age                   0
growth                0
attacking             0
crossing              0
finishing             0
heading accuracy      0
short passing         0
volleys              41
skill                 0
dribbling             0
curve                41
fk accuracy           0
long passing          0
ball control          0
movement              0
acceleration          0
sprint speed          0
agility              41
reactions             0
balance              41
power                 0
shot power            0
jumping              41
stamina               0
strength              0
long shots            0
mentality             0
aggression            0
interceptions         7
positioning           7
vision               41
penalties             0
composure           279
defending             0
marking               0
standing tackle       0
sliding tackle       41
goalkeeping           0
gk diving             0
gk handling           0
gk kicking      

In [21]:
#REPLACE THE NaN WITH THE AVERAGE OF ALL PLAYERS IN SIMILAR POSITION
data['position'].value_counts()

CB           1352
ST           1267
GK           1040
CDM CM        515
CM            449
             ... 
RB CM CAM       1
CF RM RW        1
LB LWB RB       1
LB CM LW        1
RW LM CAM       1
Name: position, Length: 543, dtype: int64

In [22]:
numeric['jumping'].isna().sum()

41

In [23]:
# GET A LIST OF ALL COLS WITH NaN VALUES IN THE NUMERIC DATAFRAME
NaN_col_list = []
for col in numeric.columns:
    if numeric[col].isna().sum() != 0:
        NaN_col_list.append(col)
NaN_col_list        

['volleys',
 'curve',
 'agility',
 'balance',
 'jumping',
 'interceptions',
 'positioning',
 'vision',
 'composure',
 'sliding tackle']

In [24]:
#REPLACE ALL NaN VALUES WITH THE MEAN
for colx in NaN_col_list:
    mean = np.mean(numeric[colx])
    mean = math.ceil(mean)
    #display(mean)
    numeric[colx] = numeric[colx].fillna(mean)

C:\Users\KIMERA\AppData\Local\Temp/ipykernel_20324/252384581.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric[colx] = numeric[colx].fillna(mean)


In [25]:
numeric.isna().sum()

id                  0
age                 0
growth              0
attacking           0
crossing            0
finishing           0
heading accuracy    0
short passing       0
volleys             0
skill               0
dribbling           0
curve               0
fk accuracy         0
long passing        0
ball control        0
movement            0
acceleration        0
sprint speed        0
agility             0
reactions           0
balance             0
power               0
shot power          0
jumping             0
stamina             0
strength            0
long shots          0
mentality           0
aggression          0
interceptions       0
positioning         0
vision              0
penalties           0
composure           0
defending           0
marking             0
standing tackle     0
sliding tackle      0
goalkeeping         0
gk diving           0
gk handling         0
gk kicking          0
gk positioning      0
gk reflexes         0
total stats         0
base stats

In [26]:
categorical.columns

Index(['name', 'nationality', 'club', 'bp', 'position', 'team & contract',
       'height', 'weight', 'foot', 'joined', 'value', 'wage', 'release clause',
       'contract', 'w/f', 'sm', 'a/w', 'd/w', 'ir', 'hits', 'ls', 'st', 'rs',
       'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm',
       'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb',
       'rcb', 'rb', 'gk'],
      dtype='object')

In [27]:
# GET ONLY RELEVANT COLUMNS FROM THE CATEGORICAL DATAFRAMES
# ALOT OF COLUMNS IN THE DATAFRAME ARE NOT RELEVANT TO THE PLAYER'S OVERALL RATING
# COLUMNS LIKE CLUB, NATIONALITY, TEAM & COUNTRY HAVE NO IMPACT ON THE PLAYER'S OVERALL RATINGS
categorical = categorical[['position','height','weight','foot']]

In [28]:
# CHECKING COLUMN UNIQUE VALUES
for coln in categorical.columns:
    display(categorical[coln].value_counts())

CB           1640
ST           1267
GK           1040
CDM CM        515
CM            449
             ... 
RB CM CAM       1
CF RM RW        1
LB LWB RB       1
LB CM LW        1
RW LM CAM       1
Name: position, Length: 543, dtype: int64

6'0"     1751
5'10"    1636
5'9"     1401
5'11"    1314
6'2"     1261
6'1"     1257
6'3"      659
5'8"      657
5'7"      614
6'4"      497
5'6"      209
6'5"      178
5'5"      129
6'6"       75
5'4"       19
6'7"       19
6'8"       10
5'3"        9
5'2"        3
5'1"        2
6'9"        1
Name: height, dtype: int64

154lbs    931
165lbs    871
176lbs    694
159lbs    651
172lbs    626
161lbs    605
163lbs    573
168lbs    555
170lbs    506
150lbs    453
157lbs    425
174lbs    406
181lbs    396
143lbs    338
183lbs    332
152lbs    324
179lbs    312
148lbs    295
185lbs    271
187lbs    262
146lbs    244
141lbs    207
190lbs    175
194lbs    159
139lbs    150
192lbs    125
198lbs    110
137lbs    106
132lbs    105
196lbs     91
134lbs     82
201lbs     56
203lbs     48
205lbs     33
209lbs     32
128lbs     29
207lbs     26
130lbs     19
126lbs     11
212lbs     11
214lbs     10
123lbs      9
216lbs      8
121lbs      7
220lbs      5
225lbs      4
218lbs      3
227lbs      2
223lbs      2
117lbs      1
236lbs      1
243lbs      1
229lbs      1
119lbs      1
234lbs      1
Name: weight, dtype: int64

Right    8828
Left     2873
Name: foot, dtype: int64

In [29]:
# PLAYERS HAVE MULTIPLE POSITIONS. FROM LOOKING AT THE DATA, THE FIRST ONE IS THEIR NATURAL/BEST POSITION
categorical['position'] = categorical['position'].apply(lambda x: x.split(' ')[0])
categorical['position']
display(categorical['position'].value_counts())

CB      2139
ST      1803
CM      1310
GK      1040
CDM      878
LB       839
RB       821
CAM      759
LM       687
RM       667
LW       295
RW       271
CF        87
RWB       53
LWB       48
ACB        1
IVST       1
IIRM       1
ND         1
Name: position, dtype: int64

In [30]:
#REMOVE ALL PLAYERS WHOSE POSITIONS DO NOT APPEAR AMONG THE PROVIDED POSITIONS
#REASON: They are few - 4 players in total

drop_list = ['ACB','IVST','IIRM','ND']
for item in drop_list:
    categorical = categorical.drop(categorical.loc[categorical['position']==item].index)
display(categorical['position'].value_counts())

CB     2139
ST     1803
CM     1310
GK     1040
CDM     878
LB      839
RB      821
CAM     759
LM      687
RM      667
LW      295
RW      271
CF       87
RWB      53
LWB      48
Name: position, dtype: int64

In [31]:
# THE COLUMNS NEED TO BE TREATED DIFFERENTLY. FOOT HAS TWO UNIQUE VALUES
# WE NEED TO CONVERT WEIGHT TO INT OR FLOAT BECAUSE A PLAYER'S WEIGHT IS A NUMBER
# CONVERT THE PLAYER'S HEIGHT TO METERS
# PLAYER ARE GIVEN VARIOUS POSITIONAL COMBINTATIONS. I WILL TAKE THE FIRST POSITION AS THE PLAYER'S POSITION

In [32]:
# CONVERT WEIGHT TO INTEGER BY REMOVING THE 'lbs' AT THE END AND CONVERT STRING INTO INT 

categorical['weight'] = categorical['weight'].apply(lambda x: int(x.replace('lbs', '')))
categorical['weight']

0        161
1        159
2        134
3        152
4        150
        ... 
11696    168
11697    174
11698    143
11699    179
11700    143
Name: weight, Length: 11697, dtype: int64

In [33]:
# CONVERT THE HEIGHT FROM STRING TO METERS
ureg = pint.UnitRegistry()
def replace_char(x):    
    x = re.sub('[^a-zA-Z0-9 \n\.]', '', x)
    if len(x) > 2:             
        y = int(x[0]) * ureg.feet
        z = int(x[-2]) * ureg.inch
        x = round(y.to(ureg.meter) + z.to(ureg.meter),2)
    
    elif len(x) == 2:
        y = int(x[0])* ureg.feet
        z = int(x[-1]) * ureg.inch
        x = round(y.to(ureg.meter) + z.to(ureg.meter),2)
    return x.magnitude
    #REMOVES THE meter AND RETURN THE MAGNITUDE ONLY

categorical['height'] = categorical['height'].apply(lambda x: replace_char(x))
categorical['height'].head()

0    1.75
1    1.83
2    1.63
3    1.55
4    1.55
Name: height, dtype: float64

In [34]:
print(categorical.columns)
categorical['foot'].value_counts()

Index(['position', 'height', 'weight', 'foot'], dtype='object')


Right    8826
Left     2871
Name: foot, dtype: int64

In [35]:
categorical['foot'].head()

0    Right
1    Right
2    Right
3    Right
4    Right
Name: foot, dtype: object

In [36]:
categorical.columns

Index(['position', 'height', 'weight', 'foot'], dtype='object')

In [37]:
# ENCODING CATEGORICAL DATA USING ONE ENCODER
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder().fit(categorical[['position','foot']])

#print(encoder.categories) - DOES NOT WORK. WHY???
encoded = encoder.transform(categorical[['position','foot']]).toarray()

# encoder.categories_ IS A NESTED ARRAY
#WE CAN FLATTEN AN ARRAY OF ARRAYS USING LISTS COMPREHESION
#Link: https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists
final_cols = [item for sublist in encoder.categories_ for item in sublist]
print(final_cols)
encoded = encoder.transform(categorical[['position','foot']]).toarray()
onehot_encoded = pd.DataFrame(encoded,columns=final_cols)
onehot_encoded.head()

['CAM', 'CB', 'CDM', 'CF', 'CM', 'GK', 'LB', 'LM', 'LW', 'LWB', 'RB', 'RM', 'RW', 'RWB', 'ST', 'Left', 'Right']


,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,Left,Right
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [38]:
# HEIGHT AND WEIGHT ARE NUMERICAL VALUES. NO NEED TO ENCODE THEM. JUST ADD THEM TO THE DATAFRAME
# MAKE THE FINAL DATAFRAME
data = pd.concat([numeric, onehot_encoded, categorical[['weight','height']]], axis = 1)
data.head()

,id,age,growth,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,...,LWB,RB,RM,RW,RWB,ST,Left,Right,weight,height
0,184383,26,1,258,54,47,43,70,44.0,286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,161.0,1.75
1,188044,30,0,365,66,79,76,68,76.0,375,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,159.0,1.83
2,184431,33,0,336,73,76,34,78,75.0,424,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,134.0,1.63
3,233796,22,13,242,44,42,58,62,36.0,259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,152.0,1.55
4,234799,23,8,249,49,37,61,68,34.0,280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,150.0,1.55


In [39]:
# DROP COLUMN THAT DO NOT HAVE AN IMPACT ON THE PLAYER'S RATING
# AGE, ID
data = data.drop(['id', 'age'], axis = 1)
data.columns

Index(['growth', 'attacking', 'crossing', 'finishing', 'heading accuracy',
       'short passing', 'volleys', 'skill', 'dribbling', 'curve',
       'fk accuracy', 'long passing', 'ball control', 'movement',
       'acceleration', 'sprint speed', 'agility', 'reactions', 'balance',
       'power', 'shot power', 'jumping', 'stamina', 'strength', 'long shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing tackle',
       'sliding tackle', 'goalkeeping', 'gk diving', 'gk handling',
       'gk kicking', 'gk positioning', 'gk reflexes', 'total stats',
       'base stats', 'pac', 'sho', 'pas', 'dri', 'def', 'phy', 'ova', 'CAM',
       'CB', 'CDM', 'CF', 'CM', 'GK', 'LB', 'LM', 'LW', 'LWB', 'RB', 'RM',
       'RW', 'RWB', 'ST', 'Left', 'Right', 'weight', 'height'],
      dtype='object')

In [40]:
data.isna().sum()

growth              0
attacking           0
crossing            0
finishing           0
heading accuracy    0
                   ..
ST                  4
Left                4
Right               4
weight              4
height              4
Length: 70, dtype: int64

In [41]:
# GET A LIST OF ALL COLS WITH NaN VALUES IN THE OVERALL DATAFRAME
NaN_col_list = []
for col in data.columns:
    if data[col].isna().sum() != 0:
        NaN_col_list.append(col)
NaN_col_list 

['CAM',
 'CB',
 'CDM',
 'CF',
 'CM',
 'GK',
 'LB',
 'LM',
 'LW',
 'LWB',
 'RB',
 'RM',
 'RW',
 'RWB',
 'ST',
 'Left',
 'Right',
 'weight',
 'height']

In [42]:
# SINCE ALL VALUES IN THE DATAFRAME ARE NUMERICAL, REPLACE NaN WITH THE MEAN
for colx in NaN_col_list:
    mean = np.mean(data[colx])
    mean = math.ceil(mean)
    data[colx] = data[colx].fillna(mean)

In [43]:
data.isna().sum()

growth              0
attacking           0
crossing            0
finishing           0
heading accuracy    0
                   ..
ST                  0
Left                0
Right               0
weight              0
height              0
Length: 70, dtype: int64

In [44]:
# CHECK IF THERE IS STILL A COLUMN WITH NaN
NaN_col_list = []
for col in data.columns:
    if data[col].isna().sum() != 0:
        NaN_col_list.append(col)
NaN_col_list 

[]

In [45]:
y = data['ova']
X = data.drop(['ova'], axis = 1)

In [46]:
# TRAIN TEST SPLIT BEOFRE NORMALIZING
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(9360, 69)
(2341, 69)
(9360,)
(2341,)


In [47]:
# NORMALIZE BOTH THE X_train AND X_test using MinMax Scaler

kimera = MinMaxScaler().fit(X_train)
x_train_normalized = kimera.transform(X_train)
print(x_train_normalized.shape)
x_train_normalized = pd.DataFrame(x_train_normalized,columns=X_train.columns)
x_train_normalized.head()

(9360, 69)


,growth,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,...,LWB,RB,RM,RW,RWB,ST,Left,Right,weight,height
0,0.037037,0.650990,0.517241,0.673913,0.625000,0.552941,0.717647,0.505855,0.593407,0.588889,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.411290,0.549020
1,0.111111,0.123762,0.091954,0.054348,0.125000,0.294118,0.094118,0.091335,0.065934,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.604839,0.745098
2,0.777778,0.532178,0.609195,0.402174,0.454545,0.658824,0.423529,0.552693,0.615385,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.322581,0.000000
3,0.259259,0.415842,0.252874,0.347826,0.670455,0.435294,0.294118,0.271663,0.318681,0.244444,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.459677,0.745098
4,0.481481,0.599010,0.609195,0.565217,0.409091,0.576471,0.694118,0.618267,0.725275,0.677778,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.322581,0.392157


In [48]:
# NORMALIZE X_test

kimera = MinMaxScaler().fit(X_test)
x_test_normalized = kimera.transform(X_test)
print(x_test_normalized.shape)
x_test_normalized = pd.DataFrame(x_train_normalized,columns=X_test.columns)
x_test_normalized.head()

(2341, 69)


,growth,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,...,LWB,RB,RM,RW,RWB,ST,Left,Right,weight,height
0,0.037037,0.650990,0.517241,0.673913,0.625000,0.552941,0.717647,0.505855,0.593407,0.588889,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.411290,0.549020
1,0.111111,0.123762,0.091954,0.054348,0.125000,0.294118,0.094118,0.091335,0.065934,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.604839,0.745098
2,0.777778,0.532178,0.609195,0.402174,0.454545,0.658824,0.423529,0.552693,0.615385,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.322581,0.000000
3,0.259259,0.415842,0.252874,0.347826,0.670455,0.435294,0.294118,0.271663,0.318681,0.244444,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.459677,0.745098
4,0.481481,0.599010,0.609195,0.565217,0.409091,0.576471,0.694118,0.618267,0.725275,0.677778,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.322581,0.392157


In [49]:
# TRAIN THE MODEL

lm = linear_model.LinearRegression()
lm.fit(X_train,y_train) 

LinearRegression()

In [50]:
# Applying Model
from sklearn.metrics import r2_score
predictions = lm.predict(X_train)
r2_score(y_train, predictions)
print(round(r2_score(y_train, predictions),2))

0.89


In [51]:
#EVALUATING THE MODEL USING THE TEST DATA

predictions = lm.predict(X_test)
r2_score(y_test, predictions)
print("R2_SCORE IS: ", round(r2_score(y_test, predictions),2))

R2_SCORE IS:  0.9


In [52]:
# # Model Validation
# from sklearn.metrics import mean_squared_error,mean_absolute_error

In [53]:
mse = mean_squared_error(y_test, predictions)
print("MSE: ", round(mse,2))

MSE:  4.98


In [54]:
rmse = np.sqrt(mean_squared_error(y_test,predictions))
print('rmse:',round(rmse,2))

rmse: 2.23


In [55]:
# CONCLUSION
# OUR MODEL IS RELATIVELY OKAY
# I WILL IMPROVE IT AS I LEARN MORE ABOUT MACHINE LEARNING